In [3]:
import pandas as pd
import pickle

import ipywidgets as widgets
from ipywidgets import interact

from src.data_load import load_tables, load_online_instance, load_distances
from src.filtering import flexible_filter
from src.plotting import plot_metrics_comparison, plot_gantt_labors_by_driver, plot_service_driver_distance, \
                         plot_gantt_by_services, plot_gantt_by_drivers
from src.metrics import collect_results_to_df, compute_metrics_with_moves, get_day_plotting_df, \
                        collect_hist_baseline_dfs
from src.experimentation_config import *
from src.config import *
from src.utils import collect_algo_baseline_df
from src.alpha_tuning_utils import collect_alpha_results_to_df, include_all_city

data_path = '../data'

instance = 'instAD2b'        # Options: ['instAS1', 'instRS1']

distance_type = 'osrm'              # Options: ['osrm', 'manhattan']
dist_method = 'haversine'      # Options: ['precalced', 'haversine']

optimization_obj = 'driver_distance'

directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
labors_real_df, labors_static_df, labors_dynamic_df = load_online_instance(data_path, instance, labors_raw_df)
dist_dict = {}

fechas = fechas_map(instance)

metricas = ['vt_count', 'num_drivers', "labores_por_conductor", 'driver_extra_time', 'driver_move_distance']

# Cargar resultados

In [4]:
labors_hist_df, moves_hist_df = collect_hist_baseline_dfs(data_path, instance, fechas, dist_method)
labors_hist_df = include_all_city(labors_hist_df)
moves_hist_df = include_all_city(moves_hist_df)

labors_algo_baseline_df, moves_algo_baseline_df = collect_algo_baseline_df(
    data_path, 
    instance, 
    dist_method,
    optimization_obj=optimization_obj)

labors_algo_baseline_df = include_all_city(labors_algo_baseline_df)
moves_algo_baseline_df = include_all_city(moves_algo_baseline_df)

with open(f'{data_path}/resultados/online_operation/{instance}/{dist_method}/res_algo_INSERT.pkl', "rb") as f:
    labors_algo_INSERT_df, moves_algo_INSERT_df = pickle.load(f)

labors_algo_INSERT_df = include_all_city(labors_algo_INSERT_df)
moves_algo_INSERT_df = include_all_city(moves_algo_INSERT_df)

with open(f'{data_path}/resultados/online_operation/{instance}/{dist_method}/res_algo_REACT.pkl', "rb") as f:
    labors_algo_REACT_df, moves_algo_REACT_df = pickle.load(f)

labors_algo_REACT_df = include_all_city(labors_algo_REACT_df)
moves_algo_REACT_df = include_all_city(moves_algo_REACT_df)

# Results

In [5]:
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from src.utils import get_city_name_from_code
from src.alpha_tuning_utils import add_aggregated_totals

algorithms = [
    {
        "name": "Histórico",
        "labors_df": labors_hist_df,
        "moves_df": moves_hist_df,
        "type": "historic",
        "color": "#800080",
        "visible": True,
    },
    {
        "name": "Offline",
        "labors_df": labors_algo_baseline_df,
        "moves_df": moves_algo_baseline_df,
        "type": "algorithm",
        "color": "#17BECF",
        "visible": True,
    },
    {
        "name": "INSERT",
        "labors_df": labors_algo_INSERT_df,
        "moves_df": moves_algo_INSERT_df,
        "type": "algorithm",
        "color": "#2ECC71",
        "visible": True,
    },
    {
        "name": "REACT",
        "labors_df": labors_algo_REACT_df,
        "moves_df": moves_algo_REACT_df,
        "type": "algorithm",
        "color": "#E67E22",
        "visible": True,
    }
]


def plot_metrics_comparison_dynamic(
    algorithms,
    city,
    metricas,
    dist_dict,
    fechas,
    metrics_with_gap=None,  # <-- NEW
    group_by=None,
    xaxis_mode="date"
):
    """Compares any number of algorithms/historic datasets dynamically, 
    with optional relative gap display vs baseline 'Histórico'."""

    # Default empty list if not provided
    metrics_with_gap = metrics_with_gap or []

    # Compute metrics for each algorithm
    metrics_by_algo = {}
    for algo in algorithms:
        df = compute_metrics_with_moves(
            algo["labors_df"], algo["moves_df"], fechas, dist_dict,
            workday_hours=8,
            city=city,
            assignment_type=algo["type"],
            skip_weekends=False,
            dist_method='haversine'
        )
        metrics_by_algo[algo["name"]] = df

    if isinstance(metricas, str):
        metricas = [metricas]

    # Prepare x-axis
    x_vals = pd.to_datetime(metrics_by_algo[algorithms[0]["name"]]["day"]).dt.strftime("%Y-%m-%d")

    # Create one figure per metric
    for metrica in metricas:
        fig = make_subplots(
            rows=1, cols=2,
            column_widths=[0.35, 0.65],
            subplot_titles=["Totales", "Serie diaria"]
        )

        totals = {
            algo["name"]: metrics_by_algo[algo["name"]][metrica].sum()
            for algo in algorithms
        }

        # Get baseline (Histórico)
        baseline_val = totals.get("Histórico", None)

        # --- Bar plot (left) ---
        for algo in algorithms:
            if algo.get("visible", True):
                val = totals[algo["name"]]

                # Compute percentage or absolute gap vs baseline
                if (
                    baseline_val is not None 
                    and baseline_val != 0 
                    and metrica in metrics_with_gap
                    and algo["name"] != "Histórico"
                ):
                    abs_gap = val - baseline_val
                    perc_gap = (abs_gap / baseline_val) * 100
                    text = f"{val:.0f}<br>{perc_gap:+.1f}%"
                else:
                    text = f"{val:.0f}"

                fig.add_trace(go.Bar(
                    x=[algo["name"]],
                    y=[val],
                    marker_color=algo["color"],
                    name=algo["name"],
                    text=[text],
                    textposition="auto"
                ), row=1, col=1)

        # --- Time series (right) ---
        for algo in algorithms:
            if algo.get("visible", True):
                dash_style = (
                    "solid" if algo["type"] == "historic"
                    else "dashdot" if "react" in algo["name"].lower()
                    else "dash"
                )
                fig.add_trace(go.Scatter(
                    x=x_vals,
                    y=metrics_by_algo[algo["name"]][metrica],
                    mode="lines+markers",
                    name=algo["name"],
                    line=dict(color=algo["color"], dash=dash_style, width=2),
                    marker=dict(size=7)
                ), row=1, col=2)

        # --- Layout ---
        fig.update_layout(
            height=500, width=1100,
            title=dict(
                text=f"{metrica} — {get_city_name_from_code(city)} "
                     f"({fechas[0]} a {fechas[1]})",
                x=0.5, xanchor="center"
            ),
            legend=dict(
                orientation="h",
                yanchor="top",
                y=-0.25,
                xanchor="center",
                x=0.5,
                font=dict(size=11)
            ),
            plot_bgcolor="white",
            margin=dict(t=100, b=120)
        )

        fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor="lightgray")
        fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor="lightgray")

        fig.show()


def plot_metrics_comparison_interactive_dynamic(
    algorithms, cities, metricas, dist_dict, fechas, metrics_with_gap=None
):
    """Fully dynamic interactive version for any number of algorithms."""

    city_dropdown = widgets.Dropdown(
        options=['ALL'] + sorted(list(set(cities))),
        value='ALL',
        description="Ciudad:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )

    # dynamically create visibility checkboxes
    algo_checks = {
        algo["name"]: widgets.Checkbox(value=algo.get("visible", True), description=algo["name"])
        for algo in algorithms
    }

    def _plot(city, **vis_flags):
        algo_copy = [{**algo, "visible": vis_flags[algo["name"]]} for algo in algorithms]
        plot_metrics_comparison_dynamic(
            algo_copy,
            city=city,
            metricas=metricas,
            dist_dict=dist_dict,
            fechas=fechas,
            metrics_with_gap=metrics_with_gap
        )

    interact(
        _plot,
        city=city_dropdown,
        **algo_checks
    )


metrics_with_gap = [
    "total_distance",
    "driver_move_distance",
    "labor_extra_time",
    "driver_extra_time"
]

plot_metrics_comparison_interactive_dynamic(
    algorithms,
    cities=valid_cities,
    metricas=metricas,
    dist_dict=dist_dict,
    fechas=(fechas[0], fechas[-1]),
    metrics_with_gap=metrics_with_gap
)


interactive(children=(Dropdown(description='Ciudad:', layout=Layout(width='300px'), options=('ALL', '1', '1004…

# Debugging

In [4]:
city = '126'
fecha = '2026-01-05'

## NUMBER OF LABORS

In [5]:
with open(f'{data_path}/resultados/online_operation/{instance}/{dist_method}/res_algo_ONLINE_static.pkl', "rb") as f:
    labors_algo_static_df, moves_algo_static_df = pickle.load(f)

filtered_labors_baseline = flexible_filter(labors_algo_baseline_df,
                city=city,
                schedule_date=fecha)

filtered_labors_static = flexible_filter(labors_algo_static_df,
                city=city,
                schedule_date=fecha)

filtered_labors_online = flexible_filter(labors_algo_REACT_df,
                city=city,
                schedule_date=fecha)

filtered_moves_baseline = flexible_filter(moves_algo_baseline_df,
                city=city,
                schedule_date=fecha)

filtered_moves_online = flexible_filter(moves_algo_REACT_df,
                city=city,
                schedule_date=fecha)

print(f'-------- Labors')
print(f'Total length of baseline is: {len(filtered_labors_baseline)}')
# print(f'Total length of static is: {len(filtered_labors_static)}')
print(f'Total length of online is: {len(filtered_labors_online)}')

print(f'\n-------- Moves')
print(f'Total length of is: {len(filtered_moves_baseline)}')
print(f'Total length of is: {len(filtered_moves_online)}')

-------- Labors
Total length of baseline is: 4
Total length of online is: 4

-------- Moves
Total length of is: 12
Total length of is: 12


In [ ]:
# Check INSERT 126 - JAN 9 (AD2b)

## CHECKING DISTANCES FOR BARRANQUILLA

### Labors

In [14]:
flexible_filter(
    labors_algo_baseline_df.drop(
        columns=['labor_type', 'labor_name', 'shop', 'alfred', 'labor_price', 'labor_created_at', 'labor_start_date', 'labor_end_date', 'client_type', 'paying_customer', 'state_service', 'labor_category']
    ),
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])

,service_id,labor_id,created_at,schedule_date,start_address_id,start_address_point,end_address_id,end_address_point,city,address_id,address_point,address_name,date,map_start_point,map_end_point,assigned_driver,actual_start,actual_end,dist_km
0,260440,356955,2025-12-31 12:24:39.813000-05:00,2026-01-05 09:30:00-05:00,142092.0,POINT (-74.813681 11.00757),68237.0,POINT (-74.7997145 11.0215868),126,130778.0,POINT (-74.7938234 10.9313923),Casa,2026-01-05,POINT (-74.813681 11.00757),POINT (-74.7997145 11.0215868),71852,2026-01-05 09:00:00-05:00,2026-01-05 09:48:16.212513-05:00,2.180139
1,262139,358749,2026-01-05 08:10:08.670000-05:00,2026-01-05 12:00:00-05:00,121049.0,POINT (-74.8051698 10.991633),87960.0,POINT (-74.776479 10.8865373),126,56474.0,POINT (-74.79419462825471 10.906019892143393),Casa,2026-01-05,POINT (-74.8051698 10.991633),POINT (-74.776479 10.8865373),71852,2026-01-05 11:30:00-05:00,2026-01-05 12:33:08.875226-05:00,12.098614
2,262244,358860,2026-01-05 10:43:35.786000-05:00,2026-01-05 13:00:00-05:00,68237.0,POINT (-74.7997145 11.0215868),135596.0,POINT (-74.8326697 11.00544),126,130778.0,POINT (-74.7938234 10.9313923),Casa,2026-01-05,POINT (-74.7997145 11.0215868),POINT (-74.8326697 11.00544),14946,2026-01-05 12:30:00-05:00,2026-01-05 13:21:01.814912-05:00,4.020166
3,262275,358897,2026-01-05 11:43:19.644000-05:00,2026-01-05 13:00:00-05:00,68237.0,POINT (-74.7997145 11.0215868),142092.0,POINT (-74.813681 11.00757),126,56474.0,POINT (-74.79419462825471 10.906019892143393),Casa,2026-01-05,POINT (-74.7997145 11.0215868),POINT (-74.813681 11.00757),71852,2026-01-05 13:03:36.421137-05:00,2026-01-05 13:51:52.633650-05:00,2.180139


In [16]:
flexible_filter(
    labors_algo_REACT_df.drop(
        columns=['labor_type', 'labor_name', 'shop', 'alfred', 'labor_price', 'labor_created_at', 'labor_start_date', 'labor_end_date', 'client_type', 'paying_customer', 'state_service', 'labor_category']
    ), 
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])

,service_id,labor_id,created_at,schedule_date,start_address_id,start_address_point,end_address_id,end_address_point,city,address_id,...,date,map_start_point,map_end_point,assigned_driver,actual_start,actual_end,dist_km,old_assigned_driver,old_actual_start,old_actual_end
0,260440,356955,2025-12-31 12:24:39.813000-05:00,2026-01-05 09:30:00-05:00,142092.0,POINT (-74.813681 11.00757),68237.0,POINT (-74.7997145 11.0215868),126,130778.0,...,2026-01-05,POINT (-74.813681 11.00757),POINT (-74.7997145 11.0215868),71852,2026-01-05 09:00:00-05:00,2026-01-05 09:48:16.212513-05:00,2.180139,71852,2026-01-05 09:00:00-05:00,2026-01-05 09:48:16.212513-05:00
1,262139,358749,2026-01-05 08:10:08.670000-05:00,2026-01-05 12:00:00-05:00,121049.0,POINT (-74.8051698 10.991633),87960.0,POINT (-74.776479 10.8865373),126,56474.0,...,2026-01-05,POINT (-74.8051698 10.991633),POINT (-74.776479 10.8865373),71852,2026-01-05 11:30:00-05:00,2026-01-05 12:33:08.875226-05:00,12.098614,NaN,NaT,NaT
2,262244,358860,2026-01-05 10:43:35.786000-05:00,2026-01-05 13:00:00-05:00,68237.0,POINT (-74.7997145 11.0215868),135596.0,POINT (-74.8326697 11.00544),126,130778.0,...,2026-01-05,POINT (-74.7997145 11.0215868),POINT (-74.8326697 11.00544),14946,2026-01-05 12:30:00-05:00,2026-01-05 13:21:01.814912-05:00,4.020166,NaN,NaT,NaT
3,262275,358897,2026-01-05 11:43:19.644000-05:00,2026-01-05 13:00:00-05:00,68237.0,POINT (-74.7997145 11.0215868),142092.0,POINT (-74.813681 11.00757),126,56474.0,...,2026-01-05,POINT (-74.7997145 11.0215868),POINT (-74.813681 11.00757),14946,2026-01-05 12:30:00-05:00,2026-01-05 13:18:16.212513-05:00,2.180139,NaN,NaT,NaT


### Moves

In [21]:
flexible_filter(
    moves_algo_baseline_df.drop(columns=['labor_id', 'labor_name']), 
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date','actual_start', 'actual_end'])

,service_id,labor_context_id,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
0,260440,356955_free,FREE_TIME,71852,2026-01-05 09:30:00-05:00,2026-01-05 08:42:30.774254-05:00,2026-01-05 08:42:30.774254-05:00,POINT (-74.7938234 10.9313923),POINT (-74.7938234 10.9313923),0.000000,0.0,126,2026-01-05
1,260440,356955_move,DRIVER_MOVE,71852,2026-01-05 09:30:00-05:00,2026-01-05 08:42:30.774254-05:00,2026-01-05 09:00:00-05:00,POINT (-74.7938234 10.9313923),POINT (-74.813681 11.00757),8.743548,17.5,126,2026-01-05
2,260440,356955_labor,VEHICLE_TRANSPORTATION,71852,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:48:16.212513-05:00,POINT (-74.813681 11.00757),POINT (-74.7997145 11.0215868),NaN,48.3,126,2026-01-05
3,262139,358749_free,FREE_TIME,71852,2026-01-05 12:00:00-05:00,2026-01-05 09:48:16.212513-05:00,2026-01-05 11:23:13.977986-05:00,POINT (-74.7997145 11.0215868),POINT (-74.7997145 11.0215868),0.000000,95.0,126,2026-01-05
4,262139,358749_move,DRIVER_MOVE,71852,2026-01-05 12:00:00-05:00,2026-01-05 11:23:13.977986-05:00,2026-01-05 11:30:00-05:00,POINT (-74.7997145 11.0215868),POINT (-74.8051698 10.991633),3.383517,6.8,126,2026-01-05
5,262139,358749_labor,VEHICLE_TRANSPORTATION,71852,2026-01-05 12:00:00-05:00,2026-01-05 11:30:00-05:00,2026-01-05 12:33:08.875226-05:00,POINT (-74.8051698 10.991633),POINT (-74.776479 10.8865373),NaN,63.1,126,2026-01-05
6,262244,358860_free,FREE_TIME,14946,2026-01-05 13:00:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 12:04:16.251117-05:00,POINT (-74.79419462825471 10.906019892143393),POINT (-74.79419462825471 10.906019892143393),0.000000,184.3,126,2026-01-05
7,262244,358860_move,DRIVER_MOVE,14946,2026-01-05 13:00:00-05:00,2026-01-05 12:04:16.251117-05:00,2026-01-05 12:30:00-05:00,POINT (-74.79419462825471 10.906019892143393),POINT (-74.7997145 11.0215868),12.864574,25.7,126,2026-01-05
8,262244,358860_labor,VEHICLE_TRANSPORTATION,14946,2026-01-05 13:00:00-05:00,2026-01-05 12:30:00-05:00,2026-01-05 13:21:01.814912-05:00,POINT (-74.7997145 11.0215868),POINT (-74.8326697 11.00544),NaN,51.0,126,2026-01-05
9,262275,358897_free,FREE_TIME,71852,2026-01-05 13:00:00-05:00,2026-01-05 12:33:08.875226-05:00,2026-01-05 12:33:08.875226-05:00,POINT (-74.776479 10.8865373),POINT (-74.776479 10.8865373),0.000000,0.0,126,2026-01-05


In [22]:
flexible_filter(
    moves_algo_REACT_df.drop(columns=['labor_id', 'labor_name']),
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date','actual_start', 'actual_end'])

,service_id,labor_context_id,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
0,260440,356955_free,FREE_TIME,71852,2026-01-05 09:30:00-05:00,2026-01-05 08:42:30.774254-05:00,2026-01-05 08:42:30.774254-05:00,POINT (-74.7938234 10.9313923),POINT (-74.7938234 10.9313923),0.000000,0.0,126,2026-01-05
1,260440,356955_move,DRIVER_MOVE,71852,2026-01-05 09:30:00-05:00,2026-01-05 08:42:30.774254-05:00,2026-01-05 09:00:00-05:00,POINT (-74.7938234 10.9313923),POINT (-74.813681 11.00757),8.743548,17.5,126,2026-01-05
2,260440,356955_labor,VEHICLE_TRANSPORTATION,71852,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:48:16.212513-05:00,POINT (-74.813681 11.00757),POINT (-74.7997145 11.0215868),NaN,48.3,126,2026-01-05
3,262139,358749_free,FREE_TIME,71852,2026-01-05 12:00:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 11:23:13.977986-05:00,POINT (-74.7997145 11.0215868),POINT (-74.7997145 11.0215868),0.000000,143.2,126,2026-01-05
4,262139,358749_move,DRIVER_MOVE,71852,2026-01-05 12:00:00-05:00,2026-01-05 11:23:13.977986-05:00,2026-01-05 11:30:00-05:00,POINT (-74.7997145 11.0215868),POINT (-74.8051698 10.991633),3.383517,6.8,126,2026-01-05
5,262139,358749_labor,VEHICLE_TRANSPORTATION,71852,2026-01-05 12:00:00-05:00,2026-01-05 11:30:00-05:00,2026-01-05 12:33:08.875226-05:00,POINT (-74.8051698 10.991633),POINT (-74.776479 10.8865373),NaN,63.1,126,2026-01-05
6,262244,358860_free,FREE_TIME,14946,2026-01-05 13:00:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 12:04:16.251117-05:00,POINT (-74.79419462825471 10.906019892143393),POINT (-74.79419462825471 10.906019892143393),0.000000,184.3,126,2026-01-05
9,262275,358897_free,FREE_TIME,14946,2026-01-05 13:00:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 12:21:57.580117-05:00,POINT (-74.8326697 11.00544),POINT (-74.8326697 11.00544),0.000000,202.0,126,2026-01-05
7,262244,358860_move,DRIVER_MOVE,14946,2026-01-05 13:00:00-05:00,2026-01-05 12:04:16.251117-05:00,2026-01-05 12:30:00-05:00,POINT (-74.79419462825471 10.906019892143393),POINT (-74.7997145 11.0215868),12.864574,25.7,126,2026-01-05
10,262275,358897_move,DRIVER_MOVE,14946,2026-01-05 13:00:00-05:00,2026-01-05 12:21:57.580117-05:00,2026-01-05 12:30:00-05:00,POINT (-74.8326697 11.00544),POINT (-74.7997145 11.0215868),4.020166,8.0,126,2026-01-05


,service_id,labor_id,labor_type,labor_name,labor_category,alfred,shop,created_at,schedule_date,client_type,...,address_id,address_point,address_name,date,map_start_point,map_end_point,assigned_driver,actual_start,actual_end,dist_km
0,228334,322426,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,69147,NaN,2025-12-30 10:06:06.882000-05:00,2026-01-06 08:00:00-05:00,B2B,...,123522.0,POINT (-75.5051041 10.392139),Apartamento,2026-01-06,POINT (-75.4851796 10.3964384),POINT (-75.51627789999999 10.4094262),69147,2026-01-06 07:30:00-05:00,2026-01-06 08:20:32.553911-05:00,3.695043
2,230678,324955,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,69147,NaN,2026-01-06 08:26:49.855000-05:00,2026-01-06 09:00:00-05:00,B2B,...,123522.0,POINT (-75.5051041 10.392139),Apartamento,2026-01-06,POINT (-75.51627789999999 10.4094262),POINT (-75.551964 10.4058317),69147,2026-01-06 08:30:00-05:00,2026-01-06 09:20:53.091638-05:00,3.923240
1,228337,322429,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,69147,NaN,2025-12-30 10:09:05.902000-05:00,2026-01-06 14:00:00-05:00,B2B,...,123522.0,POINT (-75.5051041 10.392139),Apartamento,2026-01-06,POINT (-75.51627789999999 10.4094262),POINT (-75.4851796 10.3964384),69147,2026-01-06 13:30:00-05:00,2026-01-06 14:20:32.553911-05:00,3.695043
3,230810,325097,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,69147,NaN,2026-01-06 14:27:37.246000-05:00,2026-01-06 17:00:00-05:00,B2B,...,123522.0,POINT (-75.5051041 10.392139),Apartamento,2026-01-06,POINT (-75.51627789999999 10.4094262),POINT (-75.47268369999999 10.5056449),69147,2026-01-06 16:30:00-05:00,2026-01-06 17:32:34.164531-05:00,11.712939


,service_id,labor_id,labor_type,labor_name,labor_category,alfred,shop,created_at,schedule_date,client_type,...,date,map_start_point,map_end_point,assigned_driver,actual_start,actual_end,dist_km,old_assigned_driver,old_actual_start,old_actual_end
0,228334,322426,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,69147,NaN,2025-12-30 10:06:06.882000-05:00,2026-01-06 08:00:00-05:00,B2B,...,2026-01-06,POINT (-75.4851796 10.3964384),POINT (-75.51627789999999 10.4094262),69147,2026-01-06 07:30:00-05:00,2026-01-06 08:20:32.553911-05:00,3.695043,69147,2026-01-06 07:30:00-05:00,2026-01-06 08:20:32.553911-05:00
1,230678,324955,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,69147,NaN,2026-01-06 08:26:49.855000-05:00,2026-01-06 09:00:00-05:00,B2B,...,2026-01-06,POINT (-75.51627789999999 10.4094262),POINT (-75.551964 10.4058317),69147,2026-01-06 08:30:00-05:00,2026-01-06 09:20:53.091638-05:00,3.923240,NaN,NaT,NaT
2,228337,322429,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,69147,NaN,2025-12-30 10:09:05.902000-05:00,2026-01-06 14:00:00-05:00,B2B,...,2026-01-06,POINT (-75.51627789999999 10.4094262),POINT (-75.4851796 10.3964384),69147,2026-01-06 13:30:00-05:00,2026-01-06 14:20:32.553911-05:00,3.695043,69147,2026-01-06 13:30:00-05:00,2026-01-06 14:20:32.553911-05:00
3,230810,325097,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,69147,NaN,2026-01-06 14:27:37.246000-05:00,2026-01-06 17:00:00-05:00,B2B,...,2026-01-06,POINT (-75.51627789999999 10.4094262),POINT (-75.47268369999999 10.5056449),69147,2026-01-06 16:30:00-05:00,2026-01-06 17:32:34.164531-05:00,11.712939,NaN,NaT,NaT


# NUMBER OF LABORS 

## Total number

-------- Labors


NameError: name 'filtered_labors_baseline' is not defined

## Unique number

In [9]:
print(f'-------- Labors')
print(f'Total length of baseline is: {len(filtered_labors_baseline['labor_id'].unique())}')
# print(f'Total length of static is: {len(filtered_labors_baseline)}')
print(f'Total length of online is: {len(filtered_labors_online['labor_id'].unique())}')

print(f'\n-------- Moves')
print(f'Total length of is: {len(filtered_moves_baseline['labor_id'].unique())}')
print(f'Total length of is: {len(filtered_moves_online['labor_id'].unique())}')

-------- Labors
Total length of baseline is: 62
Total length of online is: 62

-------- Moves
Total length of is: 62
Total length of is: 62


## Unusal labor